# Scrapping on WoS to get abstacts from papers

In [1]:
from suds.client import Client
import time
import pandas as pd
import csv
import numpy as np
import re

In [2]:
#from scraper_WoS_UpdatedVer import *
from scraper_WoS import *
from parsing import *

In [3]:
def pub_of_author(Querystring):
    results = soap.search(Querystring, offset = 1)
    if results.recordsFound > 100:
        new_records = results.records
        for i in range(int(results.recordsFound/100)):
            time.sleep(0.5)
            research = soap.search(Querystring, offset = (i+1)*100+1)
            time.sleep(0.5)
            new_records = new_records + research.records
        #results = new_records
        results.records = new_records
    return results

In [4]:
#soap = WosClient(lite=True)
#soap.connect()
soap = WosClient(user= 'SWISS10_reproj', password= 'Welcome#10 ', lite=False)
soap.connect()

Authenticated (SID: F39UzuHBKFuyRTgYqH3)


F39UzuHBKFuyRTgYqH3

In [4]:
searchnames_dismiss_auths_query = []
searchnames_dismiss_auths = []
searchnames_dismiss_univ = []
for line in open('../data/Dismis_Acad_List_cl.txt','r'): 
    QueryString = line.strip()
    #QueryString_author ="AU="+line.split(',')[1]+' '+line.split(',')[2][0]+"*"
    QueryString_author ="AU="+line.split(',')[1]+' '+line.split(',')[2]
    QueryString_aff = line.split(',')[6].replace('ERSITY','')
    author = line.split(',')[1]+' '+line.split(',')[2]
    searchnames_dismiss_auths.append(author)
    searchnames_dismiss_univ.append(QueryString_aff)
    searchnames_dismiss_auths_query.append(QueryString_author)
    
#remove header
searchnames_dismiss_auths_query = searchnames_dismiss_auths_query[1:]
searchnames_dismiss_auths = searchnames_dismiss_auths[1:]
searchnames_dismiss_univ = searchnames_dismiss_univ[1:]

searchnames_dismiss_auths_query

['AU=SARAC CEMAL',
 'AU=KOSE IBRAHIM',
 'AU=YENIDUNYA AHMET',
 'AU=MALKOC SIDDIK',
 'AU=ARASOGLU ALI',
 'AU=HIRCA NECATI',
 'AU=ACEMOGLU HAMIT',
 'AU=GURSUL FATIH',
 'AU=OZEL ERDOGAN',
 'AU=YAYLA AHMET',
 'AU=PERKMEN SERKAN',
 'AU=KOCYIGIT ALI',
 'AU=OZTUZCU SERDAR',
 'AU=KOC EMINE',
 'AU=CEBER MEHMET',
 'AU=DAGLI ARIF',
 'AU=BERTLEK SEYDI',
 'AU=BORAZAN HALE',
 'AU=SAGLAM MUSA',
 'AU=SELCUK MUSTAFA',
 'AU=GUNEYISI ERHAN',
 'AU=ASLANKURT MURAT',
 'AU=KESKIN YILDIRAY',
 'AU=ERAYMAN ALI',
 'AU=TATLI MUSTAFA',
 'AU=BALCIOGLU ERCAN',
 'AU=BARGAN HUSEYIN',
 'AU=YUCEL GERCEK',
 'AU=KASLI MEHMET',
 'AU=CACAN IHSAN',
 'AU=AYDIN NECMEDDIN',
 'AU=YILMAZ NIHAT',
 'AU=CETIN OZDEMIR',
 'AU=NARGUL VEYSEL',
 'AU=AKTAS DAVUT',
 'AU=TEKELIOGLU UMIT',
 'AU=YILMAZ NAZIRE',
 'AU=ILHAN AZIME',
 'AU=DEMIR SERVET',
 'AU=GUMUS METEHAN',
 'AU=ATASOGLU CENGIZ',
 'AU=SENGUN ABDULKADIR',
 'AU=OZ AHMET',
 'AU=TOY HATICE',
 'AU=SAKALAR CAGRI',
 'AU=PEKGOZLU ILHAN',
 'AU=YAPRAK MUSTAFA',
 'AU=KARACA ZEKI',
 'AU=CATA

In [6]:
d = {'Query':searchnames_dismiss_auths_query,'Authors':searchnames_dismiss_auths,'organisation':searchnames_dismiss_univ}

In [7]:
df = pd.DataFrame(d)

In [8]:
df

,Query,Authors,organisation
0,AU=SARAC CEMAL,SARAC CEMAL,MARMARA UNIV
1,AU=KOSE IBRAHIM,KOSE IBRAHIM,ADIYAMAN UNIV
2,AU=YENIDUNYA AHMET,YENIDUNYA AHMET,MARMARA UNIV
3,AU=MALKOC SIDDIK,MALKOC SIDDIK,INONU UNIV
4,AU=ARASOGLU ALI,ARASOGLU ALI,YUZUNCU YIL UNIV
5,AU=HIRCA NECATI,HIRCA NECATI,BARTIN UNIV
6,AU=ACEMOGLU HAMIT,ACEMOGLU HAMIT,ATATURK UNIV
7,AU=GURSUL FATIH,GURSUL FATIH,ISTANBUL UNIV
8,AU=OZEL ERDOGAN,OZEL ERDOGAN,INONU UNIV
9,AU=YAYLA AHMET,YAYLA AHMET,HARRAN UNIV


In [35]:
dismissed_dataset = pd.DataFrame()
for i in range(len(df)):
    try:
        print(i)
        if i == 2400: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
            soap = WosClient(user= 'SWISS10_reproj', password= 'Welcome#10 ', lite=False)
            soap.connect()
        QueryString = searchnames_dismiss_auths_query[i]
        results = pub_of_author(QueryString)
        time.sleep(0.5)
        Soup = BeautifulSoup(results.records,'lxml')
        dataset = construct_dataset(Soup,searchnames_dismiss_auths[i])
        dismissed_dataset=dismissed_dataset.append(dataset,ignore_index=True)
        #auth = auth.append(authors,ignore_index=True)
    except:
        print('Error on loop',QueryString)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
Error on loop AU=AKYOL OMER
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
Error on loop AU=YAVUZ MUSTAFA
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262

1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
Error on loop AU=OZMEN OZGUR
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
Error on loop AU=SAN SAIT
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
Error on loop AU=YEKELER ENSAR
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
Error on loop AU=UGUZ SELMAN
1885
1886
1887


Error on loop AU=AYDOGAN HAKAN
2365


Error on loop AU=ATIS MURAT
2366


Error on loop AU=VAROL MUHARREM
2367


...

Error on loop AU=ALPAY SAVAS
2396


Error on loop AU=ARPACI OZGUR
2397


Error on loop AU=FIDAN FATMA
2398


Error on loop AU=INCI ZERIFE
2399


Error on loop AU=ERDOGAN HASAN
2400
Authenticated (SID: E5sZPR19oC8YumA8bJE)
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
Error on loop AU=KUVAT SAMET
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
Error on loop AU=YILDIZ RAMAZAN
2564
2565
2566
2567
2568
2569
2570
2571
2572
25

In [22]:
#dismissed_dataset.organisation.apply(pd.Series)
dismissed_dataset.organisation.apply(pd.Series).merge(dismissed_dataset, left_index = True, right_index = True)

In [36]:
dismissed = dismissed_dataset.organisation.apply(pd.Series) \
    .merge(dismissed_dataset, right_index = True, left_index = True) \
    .drop(["organisation"], axis = 1) \
    .melt(id_vars = ['abstract', 'author','co-authors','headings','keywords','publish_date','pubtype','source','title'], value_name = "organisation") \
    .drop("variable", axis = 1)\
    .dropna()

In [37]:
#auth.organisation
result = pd.merge(df, dismissed, how='left', on=['organisation'])

In [41]:
R = result[result.Authors==result.author]

In [42]:
len(R)

28830

In [60]:
R.to_csv('../data/DISMISSED_final.csv')

In [5]:
dismissed_pd = pd.read_csv('../data/DISMISSED_final.csv')

In [25]:
dismissed_pd['labels'] = np.ones(len(dismissed_pd))

In [27]:
dismissed_pd.to_csv('../data/DISMISSED_final.csv')

In [28]:
len(dismissed_pd)

28830

In [7]:
co_aut = dismissed_pd['co-authors']
cnt_coauthors=0
for l in co_aut:
    cnt_coauthors += len(l)
print('Totally ' + str(cnt_coauthors) + ' guys were downloaded as coauthors')

Totally 3179347 guys were downloaded as coauthors


In [8]:
dismissed = set(dismissed_pd.Authors)

In [9]:
coauthors = set()
for l in co_aut :
    l = l.split(',')
    for author in l:
        if "[" in author :
            author=author.replace('[','')
            
        if "]" in author : 
            author=author.replace(']','')
        coauthors.add(author)

In [10]:
print('Totally we have ' + str(len(dismissed)) + ' dismissed and ' + str(len(coauthors)) + ' unique coauthors')


Totally we have 2083 dismissed and 34442 unique coauthors


In [11]:
undismissed = coauthors - dismissed

In [12]:
undismissed

{" 'ISINGOR MELEK'",
 "'YILMAZ MESUT'",
 " 'KEMALOGLU TUBA'",
 " 'SAHINOGLU-KESKEK NEDIME'",
 " 'UYSAL GAMZE'",
 " 'OZTURK HASAN TAHSIN'",
 " 'CIFTCI MEHMET ALI'",
 " 'OZKARA ESMA'",
 " 'ARAS N. MEVLUT'",
 " 'ELDEM VAHAP'",
 " 'KUT RECEP ALP'",
 " 'PARMAKSIZ ISKENDER'",
 "'DENIZ SAADET'",
 " 'EMIRZEOGLU MEHMET'",
 " 'GERIN FATMA'",
 "'GOREGEN MUSTAFA'",
 " 'BOZKURT OMER FARUK'",
 " 'BASMAK HIKMET'",
 " 'GUNDOGDU RAMAZAN'",
 " 'SELCUK HAKAN'",
 "'CICEK EMRAH'",
 "'SIMAVLI SERAP'",
 "'BOZKURT NURHAL MERCAN'",
 "'GULSEN SECAATTIN'",
 " 'DURAN ARIF'",
 "'ELLIS L. T.'",
 "'DAGGULLI MANSUR'",
 " 'ERKAYA SALIM'",
 "'ISIK UMRAN'",
 " 'ALTUNKAS AYSEGUEL'",
 " 'HASTAR ESIN'",
 " 'YAGMUR ERSEN AYDIN'",
 " 'KARAKURUM GUNHAN'",
 " 'YUKSEL FIKRET'",
 " 'GONDIM DIBSON D.'",
 " 'ATALAY HUSEYIN'",
 " 'GUNER SUKRIYE ILKAY'",
 " 'TAN ONDER'",
 " 'KAYALAR HUSNIYE'",
 " 'TAN SEMIH'",
 " 'KOTAN CETIN'",
 " 'CUMEN BULENT'",
 " 'OZEN MURAT EREN'",
 "'AKGUL MEHMET'",
 " 'KEMALOGLU MUSTAFA SERDAR'",
 " 'AKKAYA 

In [13]:
print('So, we have ' + str(len(undismissed)) + ' undismissed guys')

So, we have 34442 undismissed guys


In [14]:
searchnames_undismiss_auths_query = []
searchnames_undismiss_auths = []
searchnames_undismiss_univ = []
for undismissed_pers in undismissed: 
    QueryString_author ="AU="+undismissed_pers
    author = undismissed_pers
    searchnames_undismiss_auths.append(author)
    searchnames_undismiss_auths_query.append(QueryString_author)
    
#remove header
searchnames_undismiss_auths_query = searchnames_undismiss_auths_query
searchnames_undismiss_auths = searchnames_undismiss_auths

len(searchnames_undismiss_auths_query)

34442

In [17]:
undismissed_dataset = pd.DataFrame()
soap = WosClient(user= 'SWISS10_reproj', password= 'Welcome#10 ', lite=False)
soap.connect()
for i in range(4000):
    try : 
        print(i)
        if i%2400==0: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
            soap = WosClient(user= 'SWISS10_reproj', password= 'Welcome#10 ', lite=False)
            soap.connect()
        QueryString = searchnames_undismiss_auths_query[i]
        results = pub_of_author(QueryString)
        time.sleep(0.5)
        Soup = BeautifulSoup(results.records,'lxml')
        dataset = construct_dataset(Soup,searchnames_undismiss_auths[i])
        undismissed_dataset=undismissed_dataset.append(dataset,ignore_index=True)
        #auth = auth.append(authors,ignore_index=True)
    except:
        print('ERROR ON AUT',QueryString)

Authenticated (SID: D54UI7TwcdIGj8QAVzW)
0
Authenticated (SID: D5HaqIGeC9SrPLWj2ZN)
1
2
3
4
5
6
7
8
9
10
11
ERROR ON AUT AU= 'PARMAKSIZ ISKENDER'
12
13
14
15
16
17
18
19
20
21
22
23
24
25
ERROR ON AUT AU='ELLIS L. T.'
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
ERROR ON AUT AU='AKGUL MEHMET'
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
ERROR ON AUT AU= 'YILDIZ ABDULLAH'
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
ERROR ON AUT AU='USTAALIOGLU BALA BASAK OVEN'
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


ERROR ON AUT AU= 'KAYA O.'
1198
1199
1200
ERROR ON AUT AU= 'ERYILDIRIM BILAL'
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
ERROR ON AUT AU= 'MORENO P'
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
ERROR ON AUT AU='DEMIR MUSTAFA'
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
ERROR ON AUT AU= 'KAZERANI FARZANEH'
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
ERROR ON AUT AU= 'SANTOS XAVIER'
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
ERROR ON AUT AU='PAMUK GULSUM EMEL'
1346
1347
1348
1349
1350
1351
1

ERROR ON AUT AU= 'KARSLI VEDAT MEHMET'
1587


ERROR ON AUT AU= 'OZBAY GUNAY'
1588


ERROR ON AUT AU= 'YILMAZ OKKES'
1589


...

ERROR ON AUT AU= 'DAMAR OMER'
2398


ERROR ON AUT AU= 'DURDU YASEMIN'
2399


ERROR ON AUT AU= 'KARA ISA M.'
2400
Authenticated (SID: D4qUaTRLmByJsdYxQaE)
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
ERROR ON AUT AU= 'WAHL JANICE'
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535


ERROR ON AUT AU= 'NAVARRO L'
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
ERROR ON AUT AU= 'DEMIR MUSTAFA KEMAL'
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
ERROR ON AUT AU= 'BUDAK SALIH'
2634
2635
2636
2637
2638
2639
2640
ERROR ON AUT AU= 'TARHAN SERDAR'
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
ERROR ON AUT AU= 'ERDEM OZGUR'
2702
27

ERROR ON AUT AU= 'CETIN SA'
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
ERROR ON AUT AU= 'MEYER MICHAEL'
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863

In [29]:
undismissed_dataset['labels'] = np.zeros(len(undismissed_dataset))

In [31]:
undismissed_dataset.to_csv('../data/UNDISMISSED_final.csv')

In [22]:
len(set(undismissed_dataset.author))

3123

DISAMBIGUATION

## Parsing and Saving

TBC.